In [1]:
# !pip install -q \
#     torch \
#     transformers>=4.36.0 \
#     langchain>=0.1.0 \
#     langchain-community>=0.0.10 \
#     langchain-huggingface \
#     sentence-transformers>=2.3.0 \
#     faiss-cpu \
#     datasets \
#     pandas \
#     tqdm \
#     huggingface-hub>=0.20.0 \
#     ragatouille \
#     openai>=1.0.0

In [2]:
!pip install -q torch transformers \
    langchain langchain-community langchain-huggingface langchain-core langchain-openai langchain-text-splitters \
    sentence-transformers tqdm openpyxl \
    openai pandas datasets ragatouille \
    faiss-cpu \
    transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document as LangchainDocument
from langchain_community.llms import HuggingFaceHub
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.messages import SystemMessage

In [4]:
from tqdm.auto import tqdm
import pandas as pd
from typing import Tuple, Optional, List, Dict, Any
import datasets
import json

pd.set_option("display.max_colwidth", None)

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
ds = datasets.load_dataset("m-ric/huggingface_doc", split="train")

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

huggingface_doc.csv:   0%|          | 0.00/22.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2647 [00:00<?, ? examples/s]

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document as LangchainDocument

langchain_docs = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in langchain_docs:
    docs_processed += text_splitter.split_documents([doc])

  0%|          | 0/2647 [00:00<?, ?it/s]

In [8]:
from huggingface_hub import InferenceClient

repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm_client = InferenceClient(
    model = repo_id,
    timeout=120,
)

def call_llm(inference_client, prompt: str):
    response = inference_client.chat_completion(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1000,
        temperature=0.1
    )
    return response.choices[0].message.content

call_llm(llm_client, "This is a test context")

ValueError: You must provide an api_key to work with featherless-ai API or log in with `hf auth login`.

In [ ]:
QA_generation_prompt = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".
---
GOOD EXAMPLES:
Context: "The BERT model was released in 2018 by Google Research. It uses 12 transformer layers."
Question: When was the BERT model released?
Answer: The BERT model was released in 2018.

Context: "The batch size parameter controls how many samples are processed at once. The default value is 32."
Question: What is the default batch size value?
Answer: The default batch size value is 32.

BAD EXAMPLES TO AVOID:
Question: What does the context say about BERT? (references "context")
Question: Tell me more about this model. (too vague)
Answer: It depends on the use case. (not specific)
Answer: BERT is a very important model for NLP. (opinion, not fact from context)
---

Provide your answer as follows:

Output:::
Factoid question: (your factoid question)
Answer: (your answer to the factoid question)

Now here is the context.

Context: {context}\n
Output:::"""

print("Running successfully")

In [ ]:
import random

N_GENERATIONS = 10  #

print(f"Generating {N_GENERATIONS} QA couples...")

outputs = []
for sampled_context in tqdm(random.sample(docs_processed, N_GENERATIONS)):
    # Generate QA couple
    output_QA_couple = call_llm(
        llm_client, QA_generation_prompt.format(context=sampled_context.page_content)
    )
    try:
        question = output_QA_couple.split("Factoid question: ")[-1].split("Answer: ")[0]
        answer = output_QA_couple.split("Answer: ")[-1]
        assert len(answer) < 300, "Answer is too long"
        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": question,
                "answer": answer,
                "source_doc": sampled_context.metadata["source"],
            }
        )
    except:
        continue

In [ ]:
display(pd.DataFrame(outputs).head(1))

In [ ]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

print("Running successfully")

In [ ]:
print("Generating critique for each QA couple...")
for output in tqdm(outputs):
    evaluations = {
        "groundedness": call_llm(
            llm_client,
            question_groundedness_critique_prompt.format(
                context=output["context"], question=output["question"]
            ),
        ),
        "relevance": call_llm(
            llm_client,
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            llm_client,
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }
    try:
        for criterion, evaluation in evaluations.items():
            score, eval = (
                int(evaluation.split("Total rating: ")[-1].strip()),
                evaluation.split("Total rating: ")[-2].split("Evaluation: ")[1],
            )
            output.update(
                {
                    f"{criterion}_score": score,
                    f"{criterion}_eval": eval,
                }
            )
    except Exception as e:
        continue
display(pd.DataFrame(outputs).head(1))

In [ ]:
display(pd.DataFrame(outputs).head(1))

In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", None)

generated_questions = pd.DataFrame.from_dict(outputs)

print("Evaluation dataset before filtering:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

eval_dataset = datasets.Dataset.from_pandas(
    generated_questions, split="train", preserve_index=False
)

In [ ]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train")

In [ ]:
from langchain_core.documents import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds)
]

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: str,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of size `chunk_size` characters and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique

**Retriever - embeddings** 🗂️ - The retriever acts like an internal search engine

In [ ]:
#from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
import os
from langchain_community.vectorstores.faiss import FAISS


def load_embeddings(
    langchain_docs: List[LangchainDocument],
    chunk_size: int,
    embedding_model_name: Optional[str] = "thenlper/gte-small",
) -> FAISS:
    """
    Creates a FAISS index from the given embedding model and documents. Loads the index directly if it already exists.

    Args:
        langchain_docs: list of documents
        chunk_size: size of the chunks to split the documents into
        embedding_model_name: name of the embedding model to use

    Returns:
        FAISS index
    """
    # load embedding_model
    embedding_model = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        multi_process=True,
        model_kwargs={"device": "cuda"},
        encode_kwargs={
            "normalize_embeddings": True
        },  # set True to compute cosine similarity
    )

    # Check if embeddings already exist on disk
    index_name = (
        f"index_chunk:{chunk_size}_embeddings:{embedding_model_name.replace('/', '~')}"
    )
    index_folder_path = f"./data/indexes/{index_name}/"
    if os.path.isdir(index_folder_path):
        return FAISS.load_local(
            index_folder_path,
            embedding_model,
            distance_strategy=DistanceStrategy.COSINE,
            allow_dangerous_deserialization=True
        )

    else:
        print("Index not found, generating it...")
        docs_processed = split_documents(
            chunk_size,
            langchain_docs,
            embedding_model_name,
        )
        knowledge_index = FAISS.from_documents(
            docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
        )
        knowledge_index.save_local(index_folder_path)
        return knowledge_index

** the LLM Reader reads the retrieved documents to formulate its answer**

In [ ]:
RAG_PROMPT_TEMPLATE = """
<|system|>
Using the information contained in the context,
give a comprehensive answer to the question directly and factually.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.
---
### EXAMPLES OF EXPECTED BEHAVIOR

Example 1 (Sufficient Information):
Context: [1] The project was launched in 2021. [2] It cost $5 million to complete.
Question: When did the project start and what was the budget?
Assistant: The project launched in 2021 [1] and had a budget of $5 million [2].

Example 2 (Insufficient Information):
Context: [1] The company sells apples and oranges.
Question: How much do the bananas cost?
Assistant: I'm sorry, but the provided documents do not contain enough information to answer this question.

---
</s>
<|user|>
Context:
{context}
---
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

In [ ]:
!pwd


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename == ".env":
            print(os.path.join(dirname, filename))

In [ ]:
!pip install -U langchain-huggingface

In [ ]:
#from langchain_community.llms import HuggingFaceHub
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
import os
from pathlib import Path

# Try multiple .env locations
env_paths = [
    "/kaggle/input/env-setup/.env",
    ".env",
    str(Path.home() / ".env")
]

for env_path in env_paths:
    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        break

repo_id = "HuggingFaceH4/zephyr-7b-beta"
READER_MODEL_NAME = "zephyr-7b-beta"
HF_API_TOKEN = os.getenv("HF_API_TOKEN")
if not HF_API_TOKEN:
    raise ValueError("HF_API_TOKEN is missing. Put it in your .env file!")

# Use HuggingFaceEndpoint instead of HuggingFaceHub
llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    huggingfacehub_api_token=HF_API_TOKEN,
    task="conversational",
    max_new_tokens=512,
    temperature=0.5,
    repetition_penalty=1.1
)
READER_LLM = ChatHuggingFace(llm=llm)
print("Model initialized with HuggingFaceEndpoint")

In [ ]:
!pip install ragatouille<=0.0.10

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
from ragatouille import RAGPretrainedModel
from langchain_core.vectorstores import VectorStore
from langchain_core.language_models.llms import LLM
from typing import Tuple

def answer_with_rag(
    question: str,
    llm: LLM,
    knowledge_index: VectorStore,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 7,
) -> Tuple[str, List[LangchainDocument]]:
    """Answer a question using RAG with the given knowledge index."""
    # Gather documents with retriever
    relevant_docs = knowledge_index.similarity_search(
        query=question, k=num_retrieved_docs
    )
    relevant_docs = [doc.page_content for doc in relevant_docs]  # keep only the text

    # Optionally rerank results
    if reranker:
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join(
        [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)]
    )

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    #answer = llm(final_prompt)
    #using ChatHuggingFace, we invoke it
    response = llm.invoke(final_prompt)
    answer = response.content

    return answer, relevant_docs
    
print("Run sucessfully")

**setup a judge agent. ⚖️🤖**

In [ ]:
from langchain_core.language_models import BaseChatModel

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    llm,
    knowledge_index: VectorStore,
    output_file: str,
    reranker: Optional[RAGPretrainedModel] = None,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue

        answer, relevant_docs = answer_with_rag(
            question, llm, knowledge_index, reranker=reranker
        )
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer,
            "retrieved_docs": [doc for doc in relevant_docs],
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w") as f:
            json.dump(outputs, f)

In [ ]:
EVALUATION_PROMPT = """###Task Description:
An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
3. The output format should look as follows: \"Feedback: {{write a feedback for criteria}} [RESULT] {{an integer number between 1 and 5}}\"
4. Please do not generate any other opening, closing, and explanations. Be sure to include [RESULT] in your output.

###The instruction to evaluate:
{instruction}

###Response to evaluate:
{response}

###Reference Answer (Score 5):
{reference_answer}

###Score Rubrics:
[Is the response correct, accurate, and factual based on the reference answer?]
Score 1: The response is completely incorrect, inaccurate, and/or not factual.
Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
Score 3: The response is somewhat correct, accurate, and/or factual.
Score 4: The response is mostly correct, accurate, and factual.
Score 5: The response is completely correct, accurate, and factual.

###Feedback:"""

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import SystemMessage


evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

In [ ]:
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import SystemMessage
# from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
# from langchain_core.messages import SystemMessage
#from langchain_community.chat_models import ChatOpenAI
from langchain_community.llms import HuggingFaceHub
import re
import logging
logging.basicConfig(level=logging.WARNING)

# --- CONFIGURATION ---

HF_TOKEN = os.environ["HF_API_TOKEN"]

os.environ["OPENAI_API_KEY"] = HF_TOKEN
os.environ["OPENAI_BASE_URL"] = "https://router.huggingface.co/v1"

eval_chat_model = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    huggingfacehub_api_token=HF_TOKEN,
    model_kwargs={
        "temperature": 0.1,
        "max_new_tokens": 512,
        "do_sample": False
    }
)

evaluator_name = "Mistral-7B"

evaluation_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a fair evaluator language model."),
        HumanMessagePromptTemplate.from_template(EVALUATION_PROMPT),
    ]
)

def evaluate_answers(
    answer_path: str,
    eval_chat_model,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place."""
    answers = []
    if os.path.isfile(answer_path): 
        with open(answer_path, "r") as f:
            answers = json.load(f)

    print(f"Starting evaluation with {evaluator_name}...")

    # We use enumerate to ensure we can save progress safely
    for i, experiment in tqdm(enumerate(answers), total=len(answers)):
        # Skip if already evaluated by this specific evaluator
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = evaluation_prompt_template.format_messages(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        
        try:
            # Invoke the Llama model via HF
            eval_result = eval_chat_model.invoke(eval_prompt)
            content = eval_result.content
            
            # --- 2. ROBUST PARSING LOGIC ---
            if "[RESULT]" in content:
                #feedback, score = [item.strip() for item in content.split("[RESULT]")]
                match = re.search(r"RESULTRESULT\s*(\d+)", content)
                score = match.group(1) if match else "ERROR"
            else:
                # Fallback: Assume the whole output is feedback, score is missing/error
                feedback = content
                score = "ERROR_PARSING"
                logging.warning(f"Missing [RESULT] at item {i}")

            experiment[f"eval_score_{evaluator_name}"] = score
            experiment[f"eval_feedback_{evaluator_name}"] = feedback

        except Exception as e:
            logging.error(f"Error item {i}: {e}")
            continue
            
        with open(answer_path, "w") as f:
            json.dump(answers, f, indent=4) # indent=4 makes it readable
            
print("Run successfully")

In [ ]:
from huggingface_hub import InferenceClient

# Replace this section:
# eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0, openai_api_key=OPENAI_API_KEY)
# evaluator_name = "GPT4"

# With this:
evaluator_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
eval_client = InferenceClient(model=evaluator_model_id, timeout=120)
evaluator_name = "Mistral"

def evaluate_answers(
    answer_path: str,
    eval_client: InferenceClient,
    evaluator_name: str,
    evaluation_prompt_template: ChatPromptTemplate,
) -> None:
    """Evaluates generated answers. Modifies the given answer file in place for better checkpointing."""
    answers = []
    if os.path.isfile(answer_path):
        answers = json.load(open(answer_path, "r"))

    for experiment in tqdm(answers):
        if f"eval_score_{evaluator_name}" in experiment:
            continue

        eval_prompt = EVALUATION_PROMPT.format(
            instruction=experiment["question"],
            response=experiment["generated_answer"],
            reference_answer=experiment["true_answer"],
        )
        
        eval_result = call_llm(eval_client, eval_prompt)
        
        try:
            # Try to split by [RESULT]
            if "[RESULT]" in eval_result:
                parts = eval_result.split("[RESULT]")
                feedback = parts[0].replace("Feedback:", "").strip()
                score_text = parts[1].strip()
            else:
                # Fallback: try to extract score from anywhere in the text
                feedback = eval_result
                score_text = eval_result
            
            # Extract just the number from score_text
            import re
            score_match = re.search(r'\b([1-5])\b', score_text)
            if score_match:
                score = score_match.group(1)
            else:
                score = "1"
            
            experiment[f"eval_score_{evaluator_name}"] = score
            experiment[f"eval_feedback_{evaluator_name}"] = feedback
            
        except Exception as e:
            print(f"Error parsing evaluation for question: {experiment['question'][:50]}...")
            print(f"Raw response: {eval_result[:200]}...")
            experiment[f"eval_score_{evaluator_name}"] = "1"
            experiment[f"eval_feedback_{evaluator_name}"] = f"Error: {str(e)}"

        with open(answer_path, "w") as f:
            json.dump(answers, f)

the tests and evaluate answers

In [ ]:
if not os.path.exists("./output"):
    os.mkdir("./output")

for chunk_size in [200]:  # Add other chunk sizes (in tokens) as needed
    for embeddings in ["thenlper/gte-small"]:  # Add other embeddings as needed
        for rerank in [True, False]:
            settings_name = f"chunk:{chunk_size}_embeddings:{embeddings.replace('/', '~')}_rerank:{rerank}_reader-model:{READER_MODEL_NAME}"
            output_file_name = f"./output/rag_{settings_name}.json"

            print(f"Running evaluation for {settings_name}:")

            print("Loading knowledge base embeddings...")
            knowledge_index = load_embeddings(
                RAW_KNOWLEDGE_BASE,
                chunk_size=chunk_size,
                embedding_model_name=embeddings,
            )

            print("Running RAG...")
            reranker = (
                RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
                if rerank
                else None
            )
            run_rag_tests(
                eval_dataset=eval_dataset,
                llm=READER_LLM,
                knowledge_index=knowledge_index,
                output_file=output_file_name,
                reranker=reranker,
                verbose=False,
                test_settings=settings_name,
            )

            print("Running evaluation...")
            evaluate_answers(
                output_file_name,
                #eval_chat_model,
                eval_client,
                evaluator_name,
                evaluation_prompt_template,
            )


In [ ]:
# Check what's in your JSON files
import json
import re
import glob

for file in glob.glob("./output/*.json"):
    print(f"\n{file}:")
    with open(file, 'r') as f:
        data = json.load(f)
        if len(data) > 0:
            # Check first entry
            first_entry = data[0]
            print(f"  - Total entries: {len(data)}")
            print(f"  - Keys in first entry: {first_entry.keys()}")
            print(f"  - Has eval_score_Mistral: {'eval_score_Mistral' in first_entry}")
            if 'eval_score_Mistral' in first_entry:
                print(f"  - First score value: {first_entry['eval_score_Mistral']}")
                print(f"  - First feedback sample: {first_entry.get('eval_feedback_Mistral', 'N/A')[:100]}")

**Inspect results**

In [ ]:
import re
import glob

# 1. First, check if evaluations exist
for file in glob.glob("./output/*.json"):
    with open(file, 'r') as f:
        data = json.load(f)
        has_eval = any('eval_score_Mistral' in entry for entry in data)
        print(f"{file}: Has evaluations = {has_eval}, Total entries = {len(data)}")

# 4. Process scores
def extract_score_robust(x):
    if pd.isna(x):
        return 1
    if isinstance(x, (int, float)):
        return int(x)
    
    x = str(x)
    
    patterns = [
        r'Score[:\s]+([1-5])',
        r'Total rating[:\s]+([1-5])',
        r'RESULTRESULT\s*([1-5])',
        r'\b([1-5])\b'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, x, re.IGNORECASE)
        if match:
            return int(match.group(1))
    
    return 1

In [ ]:
outputs = []
# SORT the file list to ensure deterministic order
#for file in files = sorted(glob.glob("./output/*.json"))
for file in glob.glob("./output/*.json"):
    output = pd.DataFrame(json.load(open(file, "r")))
    output["settings"] = file
    outputs.append(output)
result = pd.concat(outputs)
print(result)

In [ ]:
# result["eval_score_Mistral"] = result["eval_score_Mistral"].apply(
#     lambda x: int(x) if isinstance(x, str) else 1
# )
# result["eval_score_Mistral"] = (result["eval_score_Mistral"] - 1) / 4
result["eval_score_Mistral"] = result["eval_score_Mistral"].apply(extract_score_robust)
result["eval_score_Mistral"] = (result["eval_score_Mistral"] - 1) / 4

In [ ]:
# Examine the actual data in result DataFrame
print("Result DataFrame Info:")
print(result.columns)
print("\nFirst few rows of eval_score_Mistral:")
print(result["eval_score_Mistral"].head(10))
print("\nValue counts:")
print(result["eval_score_Mistral"].value_counts())
print("\nData types:")
print(result["eval_score_Mistral"].dtype)

In [ ]:
# Calculate averages
average_scores = result.groupby("settings")["eval_score_Mistral"].mean()
print(average_scores.sort_values())

**Example results**
Let us load the results that I obtained by tweaking the different options available in this notebook. For more detail on why these options could work or not, see the notebook on advanced_RAG.

As you can see in the graph below, some tweaks do not bring any improvement, some give huge performance boosts.

➡️ There is no single good recipe: you should try several different directions when tuning your RAG systems.

In [ ]:
import plotly.express as px

scores = datasets.load_dataset("m-ric/rag_scores_cookbook", split="train")
scores = pd.Series(scores["score"], index=scores["settings"])

In [ ]:
fig = px.bar(
    scores,
    color=scores,
    labels={
        "value": "Accuracy",
        "settings": "Configuration",
    },
    color_continuous_scale="bluered",
)
fig.update_layout(
    width=1000,
    height=600,
    barmode="group",
    yaxis_range=[0, 100],
    title="<b>Accuracy of different RAG configurations</b>",
    xaxis_title="RAG settings",
    font=dict(size=15),
)
fig.layout.yaxis.ticksuffix = "%"
fig.update_coloraxes(showscale=False)
fig.update_traces(texttemplate="%{y:.1f}", textposition="outside")
fig.show()

Faithfulness and relevant 

In [ ]:
import pandas as pd
import asyncio
from typing import List, Optional
import json
import re
from tqdm.auto import tqdm

#  Structured Output Definition (Simplified - no Pydantic needed for your setup)
def parse_eval_response(response_text: str) -> dict:
    """Extract reasoning and score from LLM response."""
    try:
        # Try to find score pattern
        score_match = re.search(r'\b([0-5])\b', response_text)
        score = int(score_match.group(1)) / 5.0 if score_match else 0.0
        
        # Extract reasoning (everything before [RESULT] or the whole text)
        if "[RESULT]" in response_text:
            reasoning = response_text.split("[RESULT]")[0].replace("Feedback:", "").strip()
        else:
            reasoning = response_text[:200]  # First 200 chars as reasoning
        
        return {"reasoning": reasoning, "score": score}
    except:
        return {"reasoning": "Parse error", "score": 0.0}


In [ ]:
# . Evaluation Prompts for Mistral 
FAITHFULNESS_PROMPT = """You are an expert RAG evaluator. Evaluate whether the response is faithful to the context.

Context:
{context}

Response to evaluate:
{response}

Question:
{question}

Task: Determine if the response is derived ONLY from the given context. Do not use outside knowledge.

Provide your evaluation in this format:
Evaluation: (your detailed reasoning)
Total rating: (a number between 1 and 5, where 1 = not faithful at all, 5 = completely faithful)

[RESULT] (just the number 1-5)"""

RELEVANCE_PROMPT = """You are an expert RAG evaluator. Evaluate whether the response is relevant to the question.

Question:
{question}

Response to evaluate:
{response}

Task: Determine if the response directly and comprehensively answers the question.

Provide your evaluation in this format:
Evaluation: (your detailed reasoning)
Total rating: (a number between 1 and 5, where 1 = not relevant at all, 5 = highly relevant)

[RESULT] (just the number 1-5)"""


In [ ]:
def run_professional_eval(eval_results_path: str, eval_client, evaluator_name: str = "Mistral"):
    """
    Run faithfulness and relevance evaluation on existing RAG results.
    
    Args:
        eval_results_path: Path to JSON file with RAG results (from run_rag_tests)
        eval_client: Your InferenceClient for evaluation
        evaluator_name: Name of evaluator model
    """
    # Load existing RAG results
    with open(eval_results_path, 'r') as f:
        rag_results = json.load(f)
    
    results = []
    print(f"Starting evaluation for {len(rag_results)} samples...")
    
    for experiment in tqdm(rag_results):
        # Skip if already evaluated
        if f"faithfulness_{evaluator_name}" in experiment and f"relevance_{evaluator_name}" in experiment:
            results.append({
                "Question": experiment['question'][:50] + "...",
                "Faithfulness": experiment[f"faithfulness_{evaluator_name}"],
                "Relevance": experiment[f"relevance_{evaluator_name}"],
                "Generated_Answer": experiment['generated_answer'][:100] + "..."
            })
            continue
        
        # Prepare context from retrieved docs
        context = "\n\n".join([f"Document {i}: {doc}" for i, doc in enumerate(experiment['retrieved_docs'])])
        
        # Evaluate Faithfulness
        faithfulness_prompt = FAITHFULNESS_PROMPT.format(
            context=context,
            response=experiment['generated_answer'],
            question=experiment['question']
        )
        faithfulness_response = call_llm(eval_client, faithfulness_prompt)
        faithfulness_result = parse_eval_response(faithfulness_response)
        
        # Evaluate Relevance
        relevance_prompt = RELEVANCE_PROMPT.format(
            question=experiment['question'],
            response=experiment['generated_answer']
        )
        relevance_response = call_llm(eval_client, relevance_prompt)
        relevance_result = parse_eval_response(relevance_response)
        
        # Store results
        experiment[f"faithfulness_score_{evaluator_name}"] = faithfulness_result['score']
        experiment[f"faithfulness_reasoning_{evaluator_name}"] = faithfulness_result['reasoning']
        experiment[f"relevance_score_{evaluator_name}"] = relevance_result['score']
        experiment[f"relevance_reasoning_{evaluator_name}"] = relevance_result['reasoning']
        
        results.append({
            "Question": experiment['question'][:50] + "...",
            "Faithfulness": faithfulness_result['score'],
            "Relevance": relevance_result['score'],
            "Generated_Answer": experiment['generated_answer'][:100] + "..."
        })
        
        # Save incrementally
        with open(eval_results_path, 'w') as f:
            json.dump(rag_results, f, indent=2)
    
    return pd.DataFrame(results)


In [ ]:

# Display Results with Styling
def display_report(df):
    """Display evaluation results with conditional formatting."""
    if len(df) == 0:
        print("No results to display")
        return None
    
    styled = df.style \
        .background_gradient(cmap='RdYlGn', subset=['Faithfulness', 'Relevance'], vmin=0, vmax=1) \
        .format({'Faithfulness': '{:.1%}', 'Relevance': '{:.1%}'}) \
        .set_caption("RAG Performance Analysis: Faithfulness vs. Relevance") \
        .set_properties(**{'text-align': 'left'}) \
        .set_table_styles([
            {'selector': 'th', 'props': [('font-size', '11pt'), ('background-color', '#f0f0f0')]},
            {'selector': 'caption', 'props': [('font-size', '14pt'), ('font-weight', 'bold')]}
        ])
    
    return styled


In [ ]:
# . Summary Statistics
def get_eval_summary(df):
    """Get summary statistics of evaluation metrics."""
    summary = {
        'Avg Faithfulness': df['Faithfulness'].mean(),
        'Avg Relevance': df['Relevance'].mean(),
        'Min Faithfulness': df['Faithfulness'].min(),
        'Max Faithfulness': df['Faithfulness'].max(),
        'Min Relevance': df['Relevance'].min(),
        'Max Relevance': df['Relevance'].max(),
        'Total Samples': len(df)
    }
    return pd.Series(summary)

# After running your RAG tests, evaluate them:
for file in glob.glob("./output/*.json"):
    print(f"\n{'='*60}")
    print(f"Evaluating: {file}")
    print('='*60)
    
    # Run professional evaluation
    results_df = run_professional_eval(file, eval_client, evaluator_name="Mistral")
    
    # Display styled report
    print("\nDetailed Results:")
    display(display_report(results_df))
    
    # Display summary
    print("\nSummary Statistics:")
    print(get_eval_summary(results_df))
    print()

In [ ]:
# Aggregate results across all settings
print("\n" + "="*60)
print("AGGREGATE RESULTS ACROSS ALL CONFIGURATIONS")
print("="*60)

all_results = []
for file in glob.glob("./output/*.json"):
    with open(file, 'r') as f:
        data = json.load(f)
    
    setting_name = file.replace('./output/rag_', '').replace('.json', '')
    
    faithfulness_scores = [d.get('faithfulness_score_Mistral', 0) for d in data if 'faithfulness_score_Mistral' in d]
    relevance_scores = [d.get('relevance_score_Mistral', 0) for d in data if 'relevance_score_Mistral' in d]
    
    if faithfulness_scores and relevance_scores:
        all_results.append({
            'Setting': setting_name,
            'Avg Faithfulness': sum(faithfulness_scores) / len(faithfulness_scores),
            'Avg Relevance': sum(relevance_scores) / len(relevance_scores),
            'Combined Score': (sum(faithfulness_scores) / len(faithfulness_scores) + sum(relevance_scores) / len(relevance_scores)) / 2
        })

aggregate_df = pd.DataFrame(all_results).sort_values('Combined Score', ascending=False)
display(aggregate_df.style.background_gradient(cmap='RdYlGn', subset=['Avg Faithfulness', 'Avg Relevance', 'Combined Score']))

Metric table


In [ ]:
# ============= CREATE COMPREHENSIVE ACCURACY METRICS TABLE =============

def create_accuracy_metrics_table():
    """
    Creates a comprehensive table showing all accuracy metrics across different RAG configurations.
    """
    all_metrics = []
    
    for file in glob.glob("./output/*.json"):
        with open(file, 'r') as f:
            data = json.load(f)
        
        if len(data) == 0:
            continue
            
        # Extract setting name
        setting_name = file.replace('./output/rag_', '').replace('.json', '')
        
        # Parse setting components
        parts = setting_name.split('_')
        chunk_size = parts[0].replace('chunk:', '')
        embeddings = parts[1].replace('embeddings:', '').replace('~', '/')
        rerank = parts[2].replace('rerank:', '')
        reader = parts[3].replace('reader-model:', '')
        
        # Calculate metrics
        metrics = {
            'Configuration': setting_name,
            'Chunk Size': chunk_size,
            'Embeddings': embeddings,
            'Reranking': rerank,
            'Reader Model': reader,
            'Total Samples': len(data)
        }
        
        # Overall Accuracy (from eval_score_Mistral or eval_score_Mixtral or eval_score_GPT4)
        for eval_name in ['Mistral', 'Mixtral', 'GPT4']:
            scores = []
            for d in data:
                score_key = f'eval_score_{eval_name}'
                if score_key in d:
                    # Extract score
                    score_val = d[score_key]
                    if isinstance(score_val, str):
                        # Try to extract number
                        import re
                        match = re.search(r'\b([1-5])\b', score_val)
                        if match:
                            scores.append(int(match.group(1)))
                        else:
                            scores.append(1)
                    elif isinstance(score_val, (int, float)):
                        scores.append(int(score_val))
            
            if scores:
                # Normalize to 0-1 scale
                normalized_scores = [(s - 1) / 4 for s in scores]
                metrics[f'Overall Accuracy ({eval_name})'] = sum(normalized_scores) / len(normalized_scores)
        
        # Faithfulness Score
        faithfulness_scores = []
        for d in data:
            for eval_name in ['Mistral', 'Mixtral']:
                key = f'faithfulness_score_{eval_name}'
                if key in d:
                    faithfulness_scores.append(d[key])
        
        if faithfulness_scores:
            metrics['Faithfulness'] = sum(faithfulness_scores) / len(faithfulness_scores)
        
        # Relevance Score
        relevance_scores = []
        for d in data:
            for eval_name in ['Mistral', 'Mixtral']:
                key = f'relevance_score_{eval_name}'
                if key in d:
                    relevance_scores.append(d[key])
        
        if relevance_scores:
            metrics['Relevance'] = sum(relevance_scores) / len(relevance_scores)
        
        # Combined Score (if both faithfulness and relevance exist)
        if 'Faithfulness' in metrics and 'Relevance' in metrics:
            metrics['Combined Score'] = (metrics['Faithfulness'] + metrics['Relevance']) / 2
        
        all_metrics.append(metrics)
    
    # Create DataFrame
    df = pd.DataFrame(all_metrics)
    
    # Sort by best overall performance
    if 'Combined Score' in df.columns:
        df = df.sort_values('Combined Score', ascending=False)
    elif any('Overall Accuracy' in col for col in df.columns):
        accuracy_col = [col for col in df.columns if 'Overall Accuracy' in col][0]
        df = df.sort_values(accuracy_col, ascending=False)
    
    return df

# Create the table
metrics_table = create_accuracy_metrics_table()

# Display with styling
def style_metrics_table(df):
    """Apply professional styling to metrics table."""
    
    # Identify numeric columns for formatting
    numeric_cols = [col for col in df.columns if any(x in col for x in ['Accuracy', 'Faithfulness', 'Relevance', 'Score'])]
    
    # Create styled dataframe
    styled = df.style
    
    # Apply gradient to numeric columns
    for col in numeric_cols:
        if col in df.columns:
            styled = styled.background_gradient(cmap='RdYlGn', subset=[col], vmin=0, vmax=1)
    
    # Format percentages
    format_dict = {col: '{:.2%}' for col in numeric_cols}
    styled = styled.format(format_dict)
    
    # Highlight best scores
    if numeric_cols:
        styled = styled.highlight_max(subset=numeric_cols, color='lightgreen', axis=0)
    
    # Set table properties
    styled = styled.set_caption("RAG System Performance Metrics - Comprehensive Evaluation") \
        .set_properties(**{'text-align': 'center'}) \
        .set_table_styles([
            {'selector': 'th', 'props': [('font-size', '12pt'), ('background-color', '#40466e'), ('color', 'white'), ('font-weight', 'bold')]},
            {'selector': 'caption', 'props': [('font-size', '16pt'), ('font-weight', 'bold'), ('margin-bottom', '10px')]},
            {'selector': 'td', 'props': [('font-size', '11pt')]},
        ])
    
    return styled

# Display the styled table
print("="*100)
print("COMPREHENSIVE RAG ACCURACY METRICS")
print("="*100)
display(style_metrics_table(metrics_table))

# ============= SUMMARY STATISTICS =============

print("\n" + "="*100)
print("SUMMARY STATISTICS")
print("="*100)

# Get numeric columns only
numeric_cols = metrics_table.select_dtypes(include=['float64', 'int64']).columns

# Calculate summary stats
summary_stats = pd.DataFrame({
    'Metric': numeric_cols,
    'Mean': [metrics_table[col].mean() for col in numeric_cols],
    'Std Dev': [metrics_table[col].std() for col in numeric_cols],
    'Min': [metrics_table[col].min() for col in numeric_cols],
    'Max': [metrics_table[col].max() for col in numeric_cols],
    'Range': [metrics_table[col].max() - metrics_table[col].min() for col in numeric_cols]
})

display(summary_stats.style.format({
    'Mean': '{:.2%}',
    'Std Dev': '{:.2%}',
    'Min': '{:.2%}',
    'Max': '{:.2%}',
    'Range': '{:.2%}'
}).background_gradient(cmap='Blues', subset=['Mean', 'Max']))

# ============= CONFIGURATION COMPARISON =============

print("\n" + "="*100)
print("IMPACT OF DIFFERENT CONFIGURATIONS")
print("="*100)

# Compare impact of reranking
if 'Reranking' in metrics_table.columns and len(metrics_table) > 0:
    rerank_comparison = metrics_table.groupby('Reranking')[numeric_cols].mean()
    print("\n Impact of Reranking:")
    display(rerank_comparison.style.format('{:.2%}').background_gradient(cmap='RdYlGn'))

# Compare impact of chunk size
if 'Chunk Size' in metrics_table.columns and len(metrics_table) > 0:
    chunk_comparison = metrics_table.groupby('Chunk Size')[numeric_cols].mean()
    print("\n Impact of Chunk Size:")
    display(chunk_comparison.style.format('{:.2%}').background_gradient(cmap='RdYlGn'))

# Compare impact of embeddings
if 'Embeddings' in metrics_table.columns and len(metrics_table) > 0:
    embedding_comparison = metrics_table.groupby('Embeddings')[numeric_cols].mean()
    print("\n Impact of Embedding Models:")
    display(embedding_comparison.style.format('{:.2%}').background_gradient(cmap='RdYlGn'))

# Compare impact of reader models
if 'Reader Model' in metrics_table.columns and len(metrics_table) > 0:
    reader_comparison = metrics_table.groupby('Reader Model')[numeric_cols].mean()
    print("\n Impact of Reader Models:")
    display(reader_comparison.style.format('{:.2%}').background_gradient(cmap='RdYlGn'))

# ============= BEST CONFIGURATION =============

print("\n" + "="*100)
print(" BEST PERFORMING CONFIGURATION")
print("="*100)

if len(metrics_table) > 0:
    # Determine best config based on available metrics
    if 'Combined Score' in metrics_table.columns:
        best_idx = metrics_table['Combined Score'].idxmax()
        metric_name = 'Combined Score'
    else:
        accuracy_cols = [col for col in metrics_table.columns if 'Overall Accuracy' in col]
        if accuracy_cols:
            metric_name = accuracy_cols[0]
            best_idx = metrics_table[metric_name].idxmax()
        else:
            best_idx = 0
            metric_name = 'Unknown'
    
    best_config = metrics_table.loc[best_idx]
    
    print(f"\n Best Configuration (by {metric_name}):")
    print(f"   Configuration: {best_config['Configuration']}")
    print(f"   Chunk Size: {best_config['Chunk Size']}")
    print(f"   Embeddings: {best_config['Embeddings']}")
    print(f"   Reranking: {best_config['Reranking']}")
    print(f"   Reader Model: {best_config['Reader Model']}")
    print()
    
    for col in numeric_cols:
        if col in best_config:
            print(f"   {col}: {best_config[col]:.2%}")

# ============= EXPORT OPTIONS =============

# Export to CSV
metrics_table.to_csv('./output/rag_accuracy_metrics.csv', index=False)
print("\n Full metrics table saved to: ./output/rag_accuracy_metrics.csv")

# Export to Excel with formatting (if openpyxl is available)
try:
    with pd.ExcelWriter('./output/rag_accuracy_metrics.xlsx', engine='openpyxl') as writer:
        metrics_table.to_excel(writer, sheet_name='Metrics', index=False)
        summary_stats.to_excel(writer, sheet_name='Summary', index=False)
    print(" Excel report saved to: ./output/rag_accuracy_metrics.xlsx")
except:
    print("  Could not save Excel file (openpyxl might not be installed)")